In [1]:
using DynamicalSystems, MAT, SharedArrays, Plots, ControlSystems

@inline @inbounds function slosh(u,v,t)

#PARAMETERS
    
    α=0.8;
    AA=1.112;
    BB=-1.531;
    E=0.5*(u[1]^2+u[2]^2)+0.5*(u[3]^2+u[4]^2);
    N1= 1.95;#v[1];
    N2=-0.25; 
    μ=4.5;
    M=(u[1]*u[4])-(u[3]*u[2]);    
      
#SISTEMA DE EQUAÇÕES Navarro(2012)
        du1=-α*u[1]-(u[5]+AA*E-2)*u[2]+BB*M*u[3];#p1 
        du2=-α*u[2]+(u[5]+AA*E+2)*u[1]+BB*M*u[4];#q1
        du3=-α*u[3]-(u[5]+AA*E-2)*u[4]-BB*M*u[1];#p2
        du4=-α*u[4]+(u[5]+AA*E+2)*u[3]-BB*M*u[2];#q2
        du5= N2-N1*u[5]-μ*(u[1]*u[2]+u[3]*u[4]); #tuning parameter, measures the offset of frequencies
  
        return SVector{5}(du1, du2, du3, du4, du5);
      
end

#CONDIÇÕES INICIAIS
      u0 = [0.1; 0.1; 1.0; 1.0; 0.0]

#INTERVALO DE VARIAÇÃO 
    inicial=-2.0;
    final=2.0;
    div=99;
    as=inicial:(final-inicial)/div:final;#

   lambda=zeros(length(as),5);

    for i=1:1:length(as)
        pp1=[as[i]];
        ds = ContinuousDynamicalSystem(slosh, u0, pp1)
        lambda[i,:]=lyapunovs(ds, 5000, Ttr=2000)
        #println(i)
    end

#  MATLAB files
 file = matopen("lyapunov.mat", "w")
 write(file, "lyap",lambda)
 close(file)

 matwrite("lyapunov.mat", 
    Dict(
 	"n1" => collect(as),
 	"lyap" => lambda,))

# GRÁFICOS

In [ ]:
ds = ContinuousDynamicalSystem(slosh, u0, collect(as));
tr = trajectory(ds, 5000;Ttr=5000);

In [ ]:
#PHASE PORTRAIT p2 x q2
default(titlefont = (14, "arial"), 
        legendfontsize = 14, 
        guidefont = (14, :black), 
        tickfont = (12, :black), 
        guide = "x", 
        framestyle = :zerolines, 
#       yminorgrid = true)
        grid=false)
plot(tr[:,3], tr[:,4], 
    seriestype = :scatter, 
    alpha = 0.5,
#     xlims=(-0.0001, 0.0001),ylims=(-0.0001, 0.0001),
    title="N1=1.45", 
    label="N1=1.45", 
    ms=0.02, 
    linewidth=0.02,
    leg=false)
    #c = :greys
plot!(xlabel="p2",ylabel="q2")
 savefig("5_FPM_145")